In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()
import fileinput
import pickle
import re
import spacy
from pandarallel import pandarallel

!python -m spacy download en_core_web_md
!python -m spacy download en_core_web_lg

In [2]:
nlp = spacy.load("en_core_web_lg")
pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [5]:
df = pd.read_pickle("p:/uni/nlp/WashingtonPost_v4/data/v5/RAWData.pkl")

In [6]:
df.Kategorie = df.Kategorie.replace("\"$|\'$", "",regex=True)
df.Titel = df.Titel.replace("\"$|\'$", "",regex=True)
df.Datum = df.Datum.replace("\"$|\'$", "",regex=True) 

In [7]:
df = df.set_index('id')


In [8]:
df.content = df.content.replace("\<.*?\>","",regex=True)

In [9]:
df["tokens"] = df.apply(lambda row: [a.text for a in list(nlp(str(row["content"])))], axis=1)

In [ ]:
all_stopwords = nlp.Defaults.stop_words
def remove_stopwords_from_list(list_in):
    return [a for a in list_in if a not in all_stopwords]

In [ ]:
#pandarallel
df["no_stop"] = df.parallel_apply(lambda row: remove_stopwords_from_list(row["tokens"]), axis=1)

df["pos"] = df.parallel_apply(lambda row: [t.pos_ for t in nlp(row["content"])], axis=1)

df["lemma"] = df.parallel_apply(lambda row: [t.lemma_ for t in nlp(row["content"])], axis=1)


In [ ]:
# no pandarallel
df["no_stop"] = df.apply(lambda row: remove_stopwords_from_list(row["tokens"]), axis=1)

df["pos"] = df.apply(lambda row: [t.pos_ for t in nlp(row["content"])], axis=1)

df["lemma"] = df.apply(lambda row: [t.lemma_ for t in nlp(row["content"])], axis=1)


In [ ]:
df

,Kategorie,Titel,Autor,Datum,content,tokens,no_stop,lemma,pos
id,,,,,,,,,
0,Education,Some students,Emma Brown,2012-01-01T22:13:59Z,Nevermind that it’s a federal holiday. And nev...,"[Nevermind, that, it, ’s, a, federal, holiday,...","[Nevermind, federal, holiday, ., And, nevermin...","[nevermind, that, it, ’, a, federal, holiday, ...","[INTJ, SCONJ, PRON, VERB, DET, ADJ, NOUN, PUNC..."
1,The Post's View,Medical experiments on humans,Editorial Board,2012-01-02T00:39:34Z,"IN 2010, THE FEDERAL government funded 55,000 ...","[IN, 2010, ,, THE, FEDERAL, government, funded...","[IN, 2010, ,, THE, FEDERAL, government, funded...","[in, 2010, ,, the, federal, government, fund, ...","[ADP, NUM, PUNCT, DET, ADJ, NOUN, VERB, NUM, N..."
2,Politics,Iowa caucuses: One day out.,Karen Tumulty; David A. Fahrenthold,2012-01-03T01:37:00Z,DES MOINES — One candidate made an appearance ...,"[DES, MOINES, —, One, candidate, made, an, app...","[DES, MOINES, —, One, candidate, appearance, w...","[DES, MOINES, —, one, candidate, make, an, app...","[PROPN, PROPN, PUNCT, NUM, NOUN, VERB, DET, NO..."
3,Technology,Rumor roundup: ‘iPad 3’ displays,Hayley Tsukayama,2012-01-03T13:58:00Z,With the new year comes a fresh burst of excit...,"[With, the, new, year, comes, a, fresh, burst,...","[With, new, year, comes, fresh, burst, excitem...","[with, the, new, year, come, a, fresh, burst, ...","[SCONJ, DET, ADJ, NOUN, VERB, DET, ADJ, NOUN, ..."
4,Technology,Nintendo 3DS hits 4M in sales,Hayley Tsukayama,2012-01-03T15:42:08Z,"Nintendo’s newest piece of hardware, the Ninte...","[Nintendo, ’s, newest, piece, of, hardware, ,,...","[Nintendo, newest, piece, hardware, ,, Nintend...","[Nintendo, ’s, new, piece, of, hardware, ,, th...","[PROPN, PART, ADJ, NOUN, ADP, NOUN, PUNCT, DET..."
5,Local,Arc of Northern Virginia provides a voice for ...,Kali Schumitz,2012-01-04T14:41:00Z,Although people of all backgrounds are struggl...,"[Although, people, of, all, backgrounds, are, ...","[Although, people, backgrounds, struggling, fi...","[although, people, of, all, background, be, st...","[SCONJ, NOUN, ADP, DET, NOUN, AUX, VERB, PART,..."
6,Technology,Yahoo: Who is new CEO Scott Thompson?,Hayley Tsukayama,2012-01-04T19:34:00Z,Yahoo named a new chief executive Wednesday mo...,"[Yahoo, named, a, new, chief, executive, Wedne...","[Yahoo, named, new, chief, executive, Wednesda...","[Yahoo, name, a, new, chief, executive, Wednes...","[PROPN, VERB, DET, ADJ, ADJ, NOUN, PROPN, NOUN..."
7,Sports,Sugar Bowl: Hokies reflect on disappointing en...,Mark Giannotto,2012-01-04T20:58:58Z,NEW ORLEANS — Virginia Tech senior wide recei...,"[NEW, ORLEANS, —, , Virginia, Tech, senior, w...","[NEW, ORLEANS, —, , Virginia, Tech, senior, w...","[NEW, ORLEANS, —, , Virginia, Tech, senior, w...","[PROPN, PROPN, PUNCT, SPACE, PROPN, PROPN, ADJ..."
8,Travel,Google’s Flight Search becoming formidable,Christopher Elliott,2012-01-05T15:15:25Z,If you haven’t Googled a flight itinerary rece...,"[If, you, have, n’t, Googled, a, flight, itine...","[If, Googled, flight, itinerary, recently, ,, ...","[if, you, have, not, google, a, flight, itiner...","[SCONJ, PRON, AUX, PART, VERB, DET, NOUN, NOUN..."
